本demo参考[廖雪峰的官方网站][1]

[1]:https://www.liaoxuefeng.com/wiki/0014316089557264a6b348958f449949df42a6d3a2e542c000/001432010494717e1db78cd172e4d52b853e7fd38d6985c000

In [1]:
# 导入SQLite驱动
import sqlite3
import os
#########多次测试需要添加的代码########
#首先除去原有的数据库test.db
db_file = os.path.join(os.getcwd(), 'test.db')
if os.path.isfile(db_file):
    os.remove(db_file)
#####################################
# 连接到SQLite数据库
# 数据库文件是test.db
# 如果文件不存在，会自动在当前目录创建
conn = sqlite3.connect('test.db')
# 创建一个Cursor
cursor = conn.cursor()
# 执行一条SQL语句，创建user表
cursor.execute('create table user (id varchar(20) primary key, name varchar(20))')

In [2]:
# 继续执行一条SQL语句，插入一条记录
cursor.execute('insert into user (id, name) values (\'1\', \'Michael\')')

In [3]:
# 通过rowcount获得插入的行数
print(cursor.rowcount)
# 关闭Cursor
cursor.close()
# 提交事务
conn.commit()
# 关闭Connection
conn.close()

1


In [4]:
conn = sqlite3.connect('test.db')
cursor = conn.cursor()
# 执行查询语句:
cursor.execute('select * from user where id=?', ('1',))

In [5]:
# 获得查询结果集:
values = cursor.fetchall()#fetchall = fetch + all
print(values)
cursor.close()
conn.close()

[('1', 'Michael')]


### 注意点

在Python中操作数据库时，要先导入数据库对应的驱动，然后，通过Connection对象和Cursor对象操作数据。

要确保打开的Connection对象和Cursor对象都正确地被关闭，否则，资源就会泄露。

如何才能确保出错的情况下也关闭掉Connection对象和Cursor对象呢？请回忆try:...except:...finally:...的用法。

### 测试：编写函数，在Sqlite中根据分数段查找指定的名字

#### 1.原有题目信息

In [6]:
import os, sqlite3

#首先除去原有的数据库test.db
db_file = os.path.join(os.getcwd(), 'test.db')
if os.path.isfile(db_file):
    os.remove(db_file)

#再新建一个数据库，也叫test.db
conn = sqlite3.connect(db_file)
cursor = conn.cursor()
cursor.execute('create table user(id varchar(20) primary key, name varchar(20), score int)')
cursor.execute(r"insert into user values ('A-001', 'Adam', 95)")
cursor.execute(r"insert into user values ('A-002', 'Bart', 62)")
cursor.execute(r"insert into user values ('A-003', 'Lisa', 78)")
cursor.close()
conn.commit()
conn.close()

#### 2. 实现函数:2个实现

In [7]:
def get_score_in1(low, high):
    ' 返回指定分数区间的名字，按分数从低到高排序 '
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()
    cursor.execute("select user.name, user.score from user where score >= ? and score <= ?", (low,high))
    res = cursor.fetchall()
    res.sort(key = lambda x: x[1])
    res = [i[0] for i in res]
    cursor.close()
    conn.commit()
    conn.close()
    return res

In [8]:

def get_score_in2(low, high):
    ' 返回指定分数区间的名字，按分数从低到高排序 '
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()
    cursor.execute("select * from user where score between ? and ? ORDER BY score", (low,high))
    res = cursor.fetchall()
    res = list(map(lambda x: x[1], res))
    cursor.close()
    conn.commit()
    conn.close()
    return res

第一种实现是我自己写的，刚学习mySQL不熟悉相关的语法，于是便自己取出数据后再sort一遍。第二种实现是看别人写的，重点就在于其**充分利用了**SQL语句，使用**ORDER BY**的方法进行排序。

#### 3.测试函数实现效果

In [9]:
assert get_score_in1(80, 95) == ['Adam']
assert get_score_in1(60, 80) == ['Bart', 'Lisa']
assert get_score_in1(60, 100) == ['Bart', 'Lisa', 'Adam']

assert get_score_in2(80, 95) == ['Adam']
assert get_score_in2(60, 80) == ['Bart', 'Lisa']
assert get_score_in2(60, 100) == ['Bart', 'Lisa', 'Adam']

print('Pass')

Pass
